In [1]:
%load_ext autoreload
%autoreload 2

import chess
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers

from chesspos.preprocessing import singlets
from chesspos.convert import board_to_bitboard,bitboard_to_board

from chesspos.ml.preprocessing.sample_generator import SampleGenerator
from chesspos.ml.models.dense_autoencoder import DenseAutoencoder
from chesspos.ml.models.cnn_autoencoder import CnnAutoencoder

In [2]:
train_generator = SampleGenerator(
    sample_dir = "/home/pafrank/Documents/coding/chess-position-embedding/data/train_medium/",
    batch_size = 32
)
train_generator.set_subsampling_functions(['singlets'])
train_generator.construct_generator()

test_generator = SampleGenerator(
    sample_dir = "/home/pafrank/Documents/coding/chess-position-embedding/data/validation_medium/",
    batch_size = 32
)
test_generator.set_subsampling_functions(['singlets'])
test_generator.construct_generator()

autoencoder = CnnAutoencoder(
    input_size = 773,
    embedding_size = 64,
    # hidden_layers = [1025, 0.3, 1024, 0.3, 512, 0.3, 512, 0.3, 256, 0.3, 256, 0.3, 256, 0.3, 128, 0.3, 128, 0.3, 128, 0.3, 128, 0.3],
    hidden_layers = [512, 0.3, 256, 0.3, 128, 0.3],
    loss = 'binary_crossentropy',
    train_generator = train_generator,
    test_generator = test_generator,
	safe_dir = "/home/pafrank/Documents/coding/chess-position-embedding/metric_learning/new_autoencoder_test/",
    train_steps_per_epoch = 100,
    test_steps_per_epoch = 20,
    tf_callbacks = [
        'checkpoints',
        keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.05, patience=10, verbose=0, mode='min', restore_best_weights=True)
    ]
)

autoencoder.build_model()
autoencoder.compile()
history = autoencoder.train()
autoencoder.save()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 773)]             0         
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 12)          0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 32)          6176      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 64)          8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 64)          0   

In [4]:
plt.plot(history.history['train_loss'])
plt.plot(history.history['val_loss'])
plt.savefig("/home/pafrank/Documents/coding/chess-position-embedding/metric_learning/new_autoencoder_test/train_loss.png")

KeyError: 'train_loss'

In [5]:
best_samples = autoencoder.get_best_samples(5, 2)
best_sample_bitboard = best_samples[0]['position'].reshape((773, -1))
best_sample_position = bitboard_to_board(best_sample_bitboard)
print(best_samples[0]['loss'])
autoencoder.compare_sample_to_prediction(best_sample_bitboard)


0.024102799594402313
Original:          Reconstructed:
r . b q k . n r    r . b q k . . r
p p . . p p b p    p p . . p p b p
. . n p . . p .    . . . . . . p .
. . . . . . . .    . . . . . . . .
. . . P . . . .    . . . . . . . .
. . N B P N . .    . . . . . N . .
P P . . . P P P    P P P . . P P P
k . B Q K . . R    k . B Q K . . R



In [7]:
worst_samples = autoencoder.get_worst_samples(5, 2)
worst_sample_bitboard = worst_samples[0]['position'].reshape((773, -1))
worst_sample_position = bitboard_to_board(worst_sample_bitboard)
print(worst_samples[0]['loss'])
autoencoder.compare_sample_to_prediction(worst_sample_bitboard)

0.16726478934288025
Original:          Reconstructed:
r . . . k . . r    r . . . . . . r
. . . . . p p .    . . . . . p p .
p q p p . b b p    . . . . . . . .
. p . . p . . .    . . . . . . . .
. Q . . P . P .    . . . . . . . .
. P . P . N . P    . . . . . . . .
N P P . . P . .    P P . . . P P P
k K . R . . . R    r . . . . . . .



In [8]:
fen = "r3r3/ppq1npkp/2p3p1/2n1p2P/4P1b1/5NP1/PPPQ1PB1/RN2R1K1 w - - 1 15"
board = chess.Board(fen)
bitboard = board_to_bitboard(board)
bitboard = bitboard.reshape((-1,773))
bitboard.shape
print(board)

r . . . r . . .
p p q . n p k p
. . p . . . p .
. . n . p . . P
. . . . P . b .
. . . . . N P .
P P P Q . P B .
R N . . R . K .


In [9]:
position_embedding = autoencoder.get_embedding_of_fen(fen)
print(position_embedding)

[[0.         0.         0.         0.         0.         0.
  0.         0.09854125 0.07537822 0.         0.         1.977425
  1.1287615  0.         1.1234765  2.5128677  0.         0.
  0.         0.         0.         0.06619611 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.8238183  0.         0.         0.12656721 0.
  0.         0.         0.         0.         0.26006028 0.
  0.21432841 0.         0.07493638 0.10044803 0.         0.
  0.34139058 0.         0.08777086 0.10652842 0.         0.
  0.         0.         0.         0.         0.         0.
  0.2474949  0.         0.         2.168271  ]]


In [11]:
reconstructed_board = simple_autoencoder.get_board_of_embedding(position_embedding)
print(reconstructed_board)

(1, 773) float32
r . . . . . . .
p p . . . p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P . . . P P P
r . . . . . K .


In [20]:
keras.backend.clear_session()

test_generator = SampleGenerator(
    sample_dir = "/home/pafrank/Documents/coding/chess-position-embedding/data/validation_small/",
    batch_size = 16
)
test_generator.set_subsampling_functions(['singlets'])
test_generator.construct_generator()
generator = test_generator.get_generator()
x, y = generator.__next__()
x = tf.random.normal((16,773))

<KerasTensor: shape=(None, 8, 8, 12) dtype=float32 (created by layer 'conv2d_transpose_1')>

In [3]:
input_layer = layers.Input(shape=773, dtype=tf.float32)

board_layer = layers.Lambda(lambda x: x[:,:768], output_shape=(768,))(input_layer)
board_layer = layers.Reshape((8,8,12))(board_layer)

metadata_layer = layers.Lambda(lambda x: x[:,768:], output_shape=(5,))(input_layer)

encoder = layers.Conv2D(32, (4, 4), activation="relu", padding="same")(board_layer)
encoder = layers.MaxPooling2D((2, 2), padding="same")(encoder)
encoder = layers.Conv2D(64, (2, 2), activation="relu", padding="same")(encoder)
encoder = layers.MaxPooling2D((2, 2), padding="same")(encoder)
encoder = layers.Flatten()(encoder)

encoder_model = keras.Model(inputs=input_layer, outputs=encoder, name='encoder')
encoder_model.summary()

decoder_input = layers.Input(shape=256)
decoder = layers.Reshape((2,2,64))(decoder_input)
decoder = layers.Conv2DTranspose(32, (2, 2), strides=2, activation="relu", padding="same")(decoder)
decoder = layers.Conv2DTranspose(12, (4, 4), strides=2, activation="relu", padding="same")(decoder)
decoder = layers.Reshape((768,))(decoder)

decoder_model = keras.Model(inputs=decoder_input, outputs=decoder, name='decoder')
decoder_model.summary()

autoencoder = encoder_model(input_layer)
autoencoder = decoder_model(autoencoder)
output_layer = layers.Concatenate()([autoencoder, metadata_layer])

autoencoder_model = keras.Model(inputs=input_layer, outputs=output_layer, name='autocoder')
autoencoder_model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 773)]             0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 768)               0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 8, 8, 12)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          6176      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0   